In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

In [4]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F

In [5]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 32
CLASS_SIZE = 10
VAL_SIZE = 10000

INFO:pytorch_lightning.utilities.seed:Global seed set to 7


In [10]:
class LitDataModule(pl.LightningDataModule):
  def __init__(self, data_dir=PATH_DATASETS, batch_size=BATCH_SIZE, val_size=VAL_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.val_size = val_size
    self.transform = {
        'train': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.7),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    }

  def prepare_data(self):
    datasets.CIFAR10(root=self.data_dir, train=True, download=True)
    datasets.CIFAR10(root=self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    if stage=='fit' or stage is None:
      cf_full = datasets.CIFAR10(root=self.data_dir, train=True, transform=self.transform['train'])
      train_size = len(cf_full) - self.val_size
      self.cf_train, self.cf_val = random_split(cf_full, [train_size, self.val_size])

    if stage=='test' or stage is None:
      self.cf_test = datasets.CIFAR10(root=self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.cf_train, batch_size=256, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.cf_val, batch_size=256, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.cf_test, batch_size=256, shuffle=False)

  def visualize(self):
    for images, _ in self.train_dataloader():
      print(f"images.shape: {images.shape}")
      plt.figure(figsize=(16,8))
      plt.axis("off")
      plt.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
      break

In [11]:
class LitResnet18():
  def __init__(self):
    super().__init__()
    self.save_hyperparameters()
    self.num_class = CLASS_SIZE

    self


  def forward(self, x):
    x = self.model(x)
    return x

  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    self.log('train loss', loss)
    return loss


  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f'{stage}_loss', loss, prog_bar=True)
      self.log(f'{stage}_acc', acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, stage="val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, stage="test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4)

In [17]:
dm = LitDataModule()
dm.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [18]:
dm.setup()

In [19]:
model = get_model()

In [20]:
trainer = pl.Trainer(max_epochs=1,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=20)],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(model, dm)

TypeError: ignored